In [352]:
import numpy as np
import pandas as pd
import datetime 
import csv
import os
import nbimporter
import torch
from torch import nn
import matplotlib.pyplot as plt

#path to curr directory 
dir_path = os.path.dirname(os.path.abspath("import_and_clean.ipynb"))

In [291]:
#mutates the data_f to replace float time info to datetime object format
def createDatetimeColumn(data_f):
    datetime_col = []
    num_rows = len(data_f.index)
    for i in range(0, num_rows):
        datetime_col += [datetime.datetime(int(data_f["#YY"][i]), int(data_f["MM"][i]), int(data_f["DD"][i]), int(data_f["hh"][i]), int(data_f["mm"][i]))]
    datetime_col = pd.DataFrame(datetime_col, columns = ['datetime'])
    data_f = data_f.drop(["#YY", "MM", "DD", "hh", "mm"], 1)
    data_f = pd.concat([data_f, datetime_col], axis = 1)
    return data_f 

#reads the raw NoAA data int
def readData(filePath):
    #read file
    data_f = pd.read_csv(filepath_or_buffer = filePath, header = 0, skiprows = [1], dtype = 'float', delim_whitespace=True, na_values = {"WVHT":"99.00", "MWD":"999.00"})    
    return data_f

#delete undesirable columns from the NoAA data for features and make datetime object
def cleanColumnsFeatures(data_f):
    cleaned_data_f = data_f[["#YY", "MM", "DD", "hh", "mm", "WVHT", "DPD", "APD", "MWD"]]
    cleaned_data_f = createDatetimeColumn(cleaned_data_f)
    cleaned_data_f = cleaned_data_f.dropna()
    cleaned_data_f = cleaned_data_f.reset_index(drop = True)
    return cleaned_data_f

#delete undesirable columns from the NoAA data for features and make datetime object
def cleanColumnsLabels(data_f):
    cleaned_data_f = data_f[["#YY", "MM", "DD", "hh", "mm", "WVHT"]]
    cleaned_data_f = createDatetimeColumn(cleaned_data_f)
    cleaned_data_f = cleaned_data_f.dropna()
    cleaned_data_f = cleaned_data_f.reset_index(drop = True)
    return cleaned_data_f
    
#min-max normalization
def min_max_normalize(data_f, col_name):
    minimum = data_f[col_name].min()
    maximum = data_f[col_name].max()
    
    data_f[col_name] = (data_f[col_name] - minimum) / (maximum - minimum)
    return data_f 


In [292]:

#input: cleaned wave data
#output: a 2d array containing all of the unique hours of observations that appear in the data and the indices]
#at which they appear
def availableDatetimes(data):
    inds = []
    n = 0
    curr_time = data["datetime"][n] 
    while n < data.shape[0]:
        if (data["datetime"][n].hour != curr_time.hour):
            inds += [n]
            curr_time = data["datetime"][n]
            n += 1
            
        elif(data["datetime"][n].hour == curr_time.hour):
            n += 1
            
    data = data.iloc[inds]
    l = len(inds)
    data = data.reset_index(drop=True)
    for i in range(0, l):
        data.at[i, "datetime"] = data.at[i, "datetime"].replace(minute=0)
    
    return data


#input: two datasets containing only the hourly info of the unique observations appearing in the data
#output: a joined array containing only the common observation times
def matchDataByTime2(d1, d2):
    d1 = availableDatetimes(d1)
    d2 = availableDatetimes(d2)
    matched = d1.merge(d2, how = 'inner', on = "datetime")
    return matched
    


In [293]:
#read data from scripps buoys for each year
tp_offshore_data = [readData(dir_path + "/data/46225h201{}.txt".format(x)) for x in range(3, 10)]
#scripps_offshore_data = [readData(dir_path + "/data/46254h201{}.txt".format(x)) for x in range(5, 10)]
scripps_nearshore_data = [readData(dir_path + "/data/ljpc1h201{}.txt".format(x)) for x in range(3, 10)]


tp_offshore_cleaned = pd.concat([cleanColumnsFeatures(x) for x in tp_offshore_data], axis = 0, ignore_index=True)
scripps_nearshore_cleaned = pd.concat([cleanColumnsLabels(x) for x in scripps_nearshore_data], axis = 0, ignore_index=True)
                            
tp_offshore_cleaned = min_max_normalize(tp_offshore_cleaned, "MWD")

In [294]:
matched_label_data = matchDataByTime2(tp_offshore_cleaned, scripps_nearshore_cleaned)

In [325]:
import torch
import numpy as np
from torchvision import datasets


#a WaveDataset class
#inputs: file_lst - a list of cleaned and matched files in the form of pandas dataframes
#, the last of which contains the desired label data in a 1D array (most likely Significant Wave Height)
#replace junk values with NaN

class WaveDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.len = data.shape[0]
        self.data = data
        
        

    def __getitem__(self, index):
        features = torch.tensor(self.data.loc[index, ["WVHT_x", "DPD", "APD", "MWD"]], requires_grad = True).float()
        label = torch.tensor(self.data.loc[index, "WVHT_y"], requires_grad = True).float()
        return features, label 

    def __len__(self):
        return self.len

    

In [343]:
import sklearn 
from sklearn.model_selection import train_test_split

#train/test split
train_set, test_set = train_test_split(matched_label_data, test_size = 0.3, train_size = 0.7)
train_set.reset_index(drop = True, inplace = True)
test_set.reset_index(drop = True, inplace = True) 

first_n = train_set[:1000].reset_index(drop=True)


In [346]:
batch_size = 10
data_loader = torch.utils.data.DataLoader(WaveDataset(train_set), batch_size = 10)

w = torch.normal(0, 0.1, size = (4, 1), requires_grad = True)
b = torch.zeros((1, 1), requires_grad = True)

def update_params_by_grad(params, learning_rate, batch_size):
    with torch.no_grad():
        for param in params:
            param -= param.grad * learning_rate / batch_size
    for param in params:
        param.grad.zero_()
    

def linreg(X, w, b):
    return torch.matmul(X, w) + b

def loss(pred_y, y):
    return (pred_y - y.reshape(pred_y.shape))**2 



num_epochs = 10
learning_rate = 0.0001
batch_size = 128


for epoch in num_epochs:
    for X, y in data_loader:  
        pred = linreg(X, w, b)
        y = y.reshape(pred.shape)
        l_vec = (pred - y)**2 
        l_vec.sum().backward()
        print(w.grad)
        print(w)
        update_params_by_grad([w, b], learning_rate, batch_size)


    

tensor([[ -18.8099],
        [-264.7054],
        [-141.6145],
        [ -11.1041]])
tensor([[ 0.0080],
        [-0.0218],
        [ 0.0038],
        [-0.0083]], requires_grad=True)
tensor([[ -24.4745],
        [-257.7175],
        [-162.5124],
        [ -13.3329]])
tensor([[ 0.0080],
        [-0.0216],
        [ 0.0039],
        [-0.0083]], requires_grad=True)
tensor([[ -12.8407],
        [-198.1842],
        [-123.1115],
        [  -8.2273]])
tensor([[ 0.0080],
        [-0.0214],
        [ 0.0041],
        [-0.0083]], requires_grad=True)
tensor([[ -17.9105],
        [-227.8812],
        [-142.9296],
        [ -11.0354]])
tensor([[ 0.0080],
        [-0.0212],
        [ 0.0042],
        [-0.0083]], requires_grad=True)
tensor([[ -18.0792],
        [-227.2414],
        [-136.0717],
        [ -11.5814]])
tensor([[ 0.0080],
        [-0.0211],
        [ 0.0043],
        [-0.0083]], requires_grad=True)
tensor([[ -30.3505],
        [-291.9934],
        [-171.3581],
        [ -14.2755]])
tenso

tensor([[ -30.4435],
        [-269.8249],
        [-160.3110],
        [ -12.1122]])
tensor([[ 0.0090],
        [-0.0106],
        [ 0.0104],
        [-0.0078]], requires_grad=True)
tensor([[ -19.0549],
        [-190.2642],
        [-109.4821],
        [  -9.4757]])
tensor([[ 0.0090],
        [-0.0103],
        [ 0.0105],
        [-0.0077]], requires_grad=True)
tensor([[ -25.9924],
        [-205.8213],
        [-126.4924],
        [ -11.1877]])
tensor([[ 0.0090],
        [-0.0102],
        [ 0.0106],
        [-0.0077]], requires_grad=True)
tensor([[ -17.1197],
        [-208.6119],
        [-122.1419],
        [  -8.4852]])
tensor([[ 0.0090],
        [-0.0100],
        [ 0.0107],
        [-0.0077]], requires_grad=True)
tensor([[ -20.0449],
        [-202.3816],
        [-126.1929],
        [ -11.0870]])
tensor([[ 0.0090],
        [-0.0099],
        [ 0.0108],
        [-0.0077]], requires_grad=True)
tensor([[ -25.1573],
        [-223.3275],
        [-157.6617],
        [ -13.1965]])
tenso

tensor([[ 0.0096],
        [-0.0035],
        [ 0.0145],
        [-0.0074]], requires_grad=True)
tensor([[ -21.5638],
        [-154.4520],
        [ -98.7816],
        [  -8.1903]])
tensor([[ 0.0096],
        [-0.0034],
        [ 0.0146],
        [-0.0074]], requires_grad=True)
tensor([[ -14.2992],
        [-175.0227],
        [-108.4266],
        [  -8.5612]])
tensor([[ 0.0096],
        [-0.0033],
        [ 0.0146],
        [-0.0074]], requires_grad=True)
tensor([[ -19.0970],
        [-151.9870],
        [ -95.9480],
        [  -8.9965]])
tensor([[ 0.0096],
        [-0.0031],
        [ 0.0147],
        [-0.0074]], requires_grad=True)
tensor([[ -12.5393],
        [-125.8514],
        [ -81.2900],
        [  -7.5679]])
tensor([[ 0.0097],
        [-0.0030],
        [ 0.0148],
        [-0.0074]], requires_grad=True)
tensor([[ -12.7003],
        [-134.0602],
        [ -86.3026],
        [  -7.5949]])
tensor([[ 0.0097],
        [-0.0029],
        [ 0.0149],
        [-0.0074]], requires_grad

tensor([[ -12.4416],
        [-111.9116],
        [ -70.7069],
        [  -7.2236]])
tensor([[ 0.0103],
        [ 0.0032],
        [ 0.0185],
        [-0.0071]], requires_grad=True)
tensor([[  -7.6592],
        [-121.1349],
        [ -65.7764],
        [  -4.4493]])
tensor([[ 0.0103],
        [ 0.0033],
        [ 0.0185],
        [-0.0071]], requires_grad=True)
tensor([[ -29.0050],
        [-189.8193],
        [-118.2878],
        [  -9.9103]])
tensor([[ 0.0103],
        [ 0.0034],
        [ 0.0186],
        [-0.0071]], requires_grad=True)
tensor([[ -22.2558],
        [-165.3758],
        [-113.0785],
        [ -10.6701]])
tensor([[ 0.0103],
        [ 0.0036],
        [ 0.0187],
        [-0.0071]], requires_grad=True)
tensor([[ -11.1634],
        [-119.2796],
        [ -71.1426],
        [  -6.2369]])
tensor([[ 0.0103],
        [ 0.0037],
        [ 0.0188],
        [-0.0071]], requires_grad=True)
tensor([[  -9.8486],
        [-106.6582],
        [ -56.4932],
        [  -4.8684]])
tenso

tensor([[ -6.5771],
        [-89.0382],
        [-51.7799],
        [ -3.7283]])
tensor([[ 0.0107],
        [ 0.0080],
        [ 0.0213],
        [-0.0068]], requires_grad=True)
tensor([[ -23.6897],
        [-151.8989],
        [ -95.2171],
        [  -8.4811]])
tensor([[ 0.0107],
        [ 0.0081],
        [ 0.0214],
        [-0.0068]], requires_grad=True)
tensor([[ -11.8499],
        [-122.0490],
        [ -79.6959],
        [  -6.3180]])
tensor([[ 0.0108],
        [ 0.0082],
        [ 0.0215],
        [-0.0068]], requires_grad=True)
tensor([[ -16.0262],
        [-140.8487],
        [ -75.9034],
        [  -8.0108]])
tensor([[ 0.0108],
        [ 0.0083],
        [ 0.0215],
        [-0.0068]], requires_grad=True)
tensor([[ -21.4710],
        [-151.5774],
        [ -98.3123],
        [  -8.9407]])
tensor([[ 0.0108],
        [ 0.0084],
        [ 0.0216],
        [-0.0068]], requires_grad=True)
tensor([[ -7.2341],
        [-97.1922],
        [-53.3778],
        [ -4.1703]])
tensor([[ 0.0

...

tensor([[ -8.5178],
        [ -3.2240],
        [-13.0254],
        [ -2.5554]])
tensor([[ 0.0217],
        [ 0.0284],
        [ 0.0409],
        [-0.0034]], requires_grad=True)
tensor([[-2.1055],
        [23.1116],
        [ 6.2567],
        [ 0.1568]])
tensor([[ 0.0217],
        [ 0.0284],
        [ 0.0409],
        [-0.0034]], requires_grad=True)
tensor([[ 2.6669],
        [55.3329],
        [35.2716],
        [ 1.8691]])
tensor([[ 0.0217],
        [ 0.0284],
        [ 0.0409],
        [-0.0034]], requires_grad=True)
tensor([[  5.6759],
        [103.1152],
        [ 63.5242],
        [  3.3575]])
tensor([[ 0.0217],
        [ 0.0284],
        [ 0.0409],
        [-0.0034]], requires_grad=True)
tensor([[ 2.3299],
        [48.2453],
        [26.8700],
        [ 1.3389]])
tensor([[ 0.0217],
        [ 0.0283],
        [ 0.0408],
        [-0.0034]], requires_grad=True)
tensor([[ 0.9711],
        [19.2861],
        [ 9.4444],
        [ 0.5285]])
tensor([[ 0.0217],
        [ 0.0283],
       

tensor([[ 0.0219],
        [ 0.0281],
        [ 0.0408],
        [-0.0033]], requires_grad=True)
tensor([[-1.9295],
        [-2.8006],
        [-1.4524],
        [-1.0213]])
tensor([[ 0.0219],
        [ 0.0281],
        [ 0.0408],
        [-0.0033]], requires_grad=True)
tensor([[-0.4755],
        [ 6.6036],
        [ 0.8751],
        [ 0.1610]])
tensor([[ 0.0219],
        [ 0.0281],
        [ 0.0408],
        [-0.0033]], requires_grad=True)
tensor([[ 1.2807],
        [66.5447],
        [30.6992],
        [ 1.8903]])
tensor([[ 0.0219],
        [ 0.0281],
        [ 0.0408],
        [-0.0033]], requires_grad=True)
tensor([[-19.1579],
        [-65.3825],
        [-50.0772],
        [ -5.1355]])
tensor([[ 0.0219],
        [ 0.0281],
        [ 0.0408],
        [-0.0033]], requires_grad=True)
tensor([[ -8.3215],
        [-47.3077],
        [-39.7204],
        [ -4.0963]])
tensor([[ 0.0219],
        [ 0.0281],
        [ 0.0408],
        [-0.0033]], requires_grad=True)
tensor([[ 4.6048],
      

tensor([[-1.1086],
        [14.3546],
        [10.0462],
        [ 0.5356]])
tensor([[ 0.0220],
        [ 0.0281],
        [ 0.0409],
        [-0.0033]], requires_grad=True)
tensor([[ 0.8216],
        [39.1997],
        [18.8351],
        [ 0.8441]])
tensor([[ 0.0220],
        [ 0.0280],
        [ 0.0409],
        [-0.0033]], requires_grad=True)
tensor([[ -6.6224],
        [-38.2380],
        [-27.4390],
        [ -3.7131]])
tensor([[ 0.0220],
        [ 0.0280],
        [ 0.0409],
        [-0.0033]], requires_grad=True)
tensor([[-1.8349],
        [ 0.8897],
        [-0.0152],
        [-0.5811]])
tensor([[ 0.0220],
        [ 0.0280],
        [ 0.0409],
        [-0.0033]], requires_grad=True)
tensor([[ 0.9249],
        [44.8711],
        [25.1010],
        [ 0.7026]])
tensor([[ 0.0220],
        [ 0.0280],
        [ 0.0409],
        [-0.0033]], requires_grad=True)
tensor([[-4.1282],
        [ 5.3991],
        [-0.6643],
        [-1.4394]])
tensor([[ 0.0220],
        [ 0.0280],
        [ 0

tensor([[-1.2156],
        [-2.6336],
        [-4.5313],
        [-0.8718]])
tensor([[ 0.0221],
        [ 0.0280],
        [ 0.0410],
        [-0.0033]], requires_grad=True)
tensor([[ 0.8311],
        [26.0087],
        [12.6454],
        [ 0.2432]])
tensor([[ 0.0221],
        [ 0.0280],
        [ 0.0410],
        [-0.0033]], requires_grad=True)
tensor([[-0.8665],
        [20.6724],
        [12.0351],
        [-0.3201]])
tensor([[ 0.0221],
        [ 0.0280],
        [ 0.0410],
        [-0.0033]], requires_grad=True)
tensor([[-14.7880],
        [-15.6919],
        [-17.3391],
        [ -2.8902]])
tensor([[ 0.0221],
        [ 0.0279],
        [ 0.0410],
        [-0.0033]], requires_grad=True)
tensor([[-16.8436],
        [-21.3511],
        [-22.1439],
        [ -3.2198]])
tensor([[ 0.0222],
        [ 0.0280],
        [ 0.0410],
        [-0.0033]], requires_grad=True)
tensor([[ 1.3149],
        [29.0759],
        [16.5439],
        [ 0.8204]])
tensor([[ 0.0222],
        [ 0.0280],
       

tensor([[ 0.0223],
        [ 0.0282],
        [ 0.0412],
        [-0.0032]], requires_grad=True)
tensor([[-6.2605],
        [-9.9808],
        [-5.4940],
        [-0.7903]])
tensor([[ 0.0223],
        [ 0.0282],
        [ 0.0412],
        [-0.0032]], requires_grad=True)
tensor([[-7.4825e-01],
        [ 1.7745e+01],
        [ 7.4949e+00],
        [-1.0514e-02]])
tensor([[ 0.0223],
        [ 0.0282],
        [ 0.0413],
        [-0.0032]], requires_grad=True)
tensor([[ 0.8895],
        [27.9212],
        [16.5003],
        [ 0.9548]])
tensor([[ 0.0223],
        [ 0.0282],
        [ 0.0412],
        [-0.0032]], requires_grad=True)
tensor([[-0.9030],
        [ 5.4274],
        [ 4.6425],
        [-0.3854]])
tensor([[ 0.0223],
        [ 0.0282],
        [ 0.0412],
        [-0.0032]], requires_grad=True)
tensor([[ -7.2556],
        [-25.0742],
        [-13.8803],
        [ -2.1624]])
tensor([[ 0.0223],
        [ 0.0282],
        [ 0.0412],
        [-0.0032]], requires_grad=True)
tensor([[ -6.

In [348]:
class LinearNet():
    
    def __init__(train_set, test_set, shape):
        self.shape = shape
        self.w = torch.normal(0, 0.1, size = (4, 1), requires_grad = True)
        self.b = torch.zeros((1, 1), requires_grad = True)
        self.train_set = train_set
        self.test_set = test_set

    def update_params_by_grad(params, learning_rate, batch_size):
        with torch.no_grad():
            for param in params:
                param -= param.grad * learning_rate / batch_size
        for param in params:
            param.grad.zero_()


    def model(X, w, b):
        return torch.matmul(X, w) + b

    #trains the model using SGD and graphs out and outputs running losses 
    def train_SGD(batch_size, learning_rate, num_epochs):  
        data_loader = torch.utils.data.DataLoader(self.train_set, batch_size = batch_size)
        epoch_losses = []
        for epoch in num_epochs:
            running_loss = 0.0
            for i, data in enumerate(data_loader):  
                X, y = data
                pred = model(X, w, b)
                y = y.reshape(pred.shape)
                l_vec = (pred - y)**2
                l = l_vec.sum()
                l.backwards()
                update_params_by_grad([w, b], learning_rate, batch_size)
                running_loss += l.item()
                
                if (i % len(data_loader)) == (len(data_loader) - 1):
                    epoch_losses.append(running_loss/len(data_loader))
                    
        plt.plot(list(range(1, num_epochs + 1)), epoch_losses)
        plt.ylabel("loss")
        plt.xlabel("epoch")
        plt.show()
        plt.savefig()
        
        
        
        
        
        return epoch_losses
                
                
                
                
    
    def test_error(): 


    

tensor([[0.0063]], requires_grad=True)